<a href="https://colab.research.google.com/github/Axargon/RemamaSensores/blob/main/Remama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.1 MB/s eta 0:00:00


In [16]:
import paho.mqtt.client as mqtt
import json
import time
import csv
from collections import deque

In [21]:
# Configurações
BROKER_ADDRESS = "test.mosquitto.org"
PORT = 1883
KEEPALIVE = 60
TOPIC = "sensores/dados"
MAX_DATA = 100  # Número máximo de dados a serem armazenados antes de salvar
CSV_FILE = "dados.csv"

# Variáveis globais
client = mqtt.Client()
fila_dados = deque(maxlen=MAX_DATA)
ultimo_json = None

# Callbacks
def on_connect(client, userdata, flags, rc):
    print(f"Conectado com código de retorno {str(rc)}")
    client.subscribe(TOPIC)

def on_message(client, userdata, msg):
    global ultimo_json
    ultimo_json = json.loads(msg.payload.decode())

# Função para obter o último JSON
def obter_ultimo_json():
    return ultimo_json

def salvar_dados_em_csv():
    global fila_dados
    fila_dados.append(obter_ultimo_json())  # Adiciona o último JSON à fila

    if len(fila_dados) == MAX_DATA:
        with open(CSV_FILE, 'a', newline='') as arquivo:
            writer = csv.writer(arquivo)
            writer.writerows(fila_dados)
            print("Últimos 100 JSONs salvos no arquivo CSV")
            fila_dados.clear()  # Limpa a fila após salvar

# Conectar ao broker
client.on_connect = on_connect
client.on_message = on_message
client.connect(BROKER_ADDRESS, PORT, KEEPALIVE)

# Loop para manter a conexão
client.loop_forever()
'''
while True:
    ultimo_json = obter_ultimo_json()
    if ultimo_json:
        print(ultimo_json)
    time.sleep(1)  # Ajuste o tempo de espera conforme necessário

'''

<ipython-input-21-3458771a26ce>:10: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Conectado com código de retorno 0


KeyboardInterrupt: 